In [1]:
import os
from crewai import Agent, Task, Crew, LLM

from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

In [3]:
llama_model = LLM(
    model="groq/llama-3.3-70b-versatile",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0
)

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llama_model
)

In [6]:
# Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llama_model
)

In [7]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
        "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llama_model
)

In [8]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

In [9]:
venue_task = Task(
    description="Find a venue in {event_city} that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
    # Outputs the venue details as a JSON file
    agent=venue_coordinator,
)

In [10]:
logistics_task = Task(
    description="Coordinate catering and "
    "equipment for an event "
    "with {expected_participants} participants "
    "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
    "including catering and equipment setup.",
    human_input=True,
    # async_execution=True,
    agent=logistics_manager,
)

In [11]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

In [12]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, logistics_manager, marketing_communications_agent],
    tasks=[venue_task, logistics_task, marketing_task],
    verbose=True,
)

In [13]:
event_details = {
    "event_topic": "Tech Innovation Conference",
    "event_description": "A gathering of tech innovators "
    "and industry leaders "
    "to explore future technologies.",
    "event_city": "San Francisco",
    "tentative_date": "2024-09-15",
    "expected_participants": 500,
    "budget": 20000,
    "venue_type": "Conference Hall",
}

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


# Agent: Venue Coordinator
## Thought: Thought: To find a suitable venue in San Francisco for the Tech Innovation Conference, I need to search for venues that can accommodate conferences and have a good reputation. I should start by searching the internet for "best conference venues in San Francisco" to get a list of potential options.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"best conference venues in San Francisco\"}"
## Tool Output: 
{'searchParameters': {'q': 'best conference venues in San Francisco', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'The 16 Best Conference Venues for Rent in San Francisco, CA', 'link': 'https://www.tagvenue.com/us/hire/conference-venues/san-francisco', 'snippet': 'Tagvenue offers an extensive range of conference venues in San Francisco, making it easy for you to find the perfec

2025-01-17 21:46:31,241 - 21652 - llm.py-llm:187 - ERROR: LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - Error code: 413 - {'error': {'message': 'Request too large for model `llama-3.3-70b-versatile` in organization `org_01hyswbryxevnt9t29p76qcc90` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 6669, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


# Agent: Venue Coordinator
## Thought: Thought: To find a suitable venue in San Francisco for the Tech Innovation Conference, I should first search the internet for venues that match the conference's criteria, such as location, capacity, and amenities. This will give me a list of potential venues to consider.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Tech Innovation Conference venues in San Francisco\"}"
## Tool Output: 
{'searchParameters': {'q': 'Tech Innovation Conference venues in San Francisco', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Top 12 tech conferences in San Francisco for 2024 - TravelPerk', 'link': 'https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/', 'snippet': 'With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights,